<a href="https://colab.research.google.com/github/icirauqui/TBot_RL/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%tensorflow_version 1.x
!pip install stable-baselines[mpi]==2.10.0

TensorFlow 1.x selected.
     |████████████████████████████████| 256kB 5.1MB/s 
  Found existing installation: stable-baselines 2.2.1
    Uninstalling stable-baselines-2.2.1:
      Successfully uninstalled stable-baselines-2.2.1


In [25]:
import pandas as pd
import os

df = pd.read_csv('data/BTCUSDT.csv')

MAX_QUOTE_ASSET_VOLUME = df.loc[
    df['Quote asset volume'].idxmax()]['Quote asset volume']

MAX_NUMBER_of_TRADES = df.loc[
    df['Number of trades'].idxmax()]['Number of trades']

MAX_TAKER_BUY_BASE_ASSET_VOLUME = df.loc[
    df['Taker buy base asset volume'].idxmax()]['Taker buy base asset volume']

MAX_TAKER_BUY_QUOTE_ASSET_VOLUME = df.loc[df[
    'Taker buy quote asset volume'].idxmax()]['Taker buy quote asset volume']

MAX_ACCOUNT_BALANCE = 10000000
INITIAL_ACCOUNT_BALANCE = 1000
MAX_CRYPTO_PRICE = 20000
MAX_CRYPTO = 21000000
MAKER_FEE = 0.00075
TAKER_FEE = 0.00075
BNBUSDTHELD = 1000
MAX_STEPS = int(os.getenv('MAX_STEPS', 400))

In [31]:
import matplotlib.pyplot as plt
import numpy as np
import gym
import random
from gym import spaces


class CryptoEnv(gym.Env):
    def __init__(self, df, title=None):
        self.df = df
        self.reward_range = (-MAX_ACCOUNT_BALANCE,
                             MAX_ACCOUNT_BALANCE)
        self.total_fees = 0
        self.total_volume_traded = 0
        self.crypto_held = 0
        self.bnb_usdt_held = BNBUSDTHELD
        self.bnb_usdt_held_start = BNBUSDTHELD
        self.episode = 1

        # Graph to render
        self.graph_reward = []
        self.graph_profit = []
        self.graph_benchmark = []

        # Action space from -1 to 1, -1 is short, 1 is buy
        self.action_space = spaces.Box(low=-1,
                                       high=1,
                                       shape=(1, ),
                                       dtype=np.float16)
        # Observation space contains only the actual price for the moment
        self.observation_space = spaces.Box(low=0,
                                            high=1,
                                            shape=(10, 5),
                                            dtype=np.float16)

    def reset(self):
        self.balance = INITIAL_ACCOUNT_BALANCE
        self.net_worth = INITIAL_ACCOUNT_BALANCE + BNBUSDTHELD
        self.max_net_worth = INITIAL_ACCOUNT_BALANCE
        BNBUSDTHELD
        self.total_fees = 0
        self.total_volume_traded = 0
        self.crypto_held = 0
        self.bnb_usdt_held = BNBUSDTHELD
        self.episode_reward = 0

        # Set the current step to a random point within the data frame
        # Weights of the current step follow the square function
        start = list(range(4, len(self.df.loc[:, 'Open'].values) - MAX_STEPS)) + self.df.index[0]
        weights = [i for i in start]
        self.current_step = random.choices(start, weights)[0]
        self.start_step = self.current_step

        return self._next_observation()

    def _next_observation(self):
        # Get the data for the last 5 timestep
        frame = np.array([
            self.df.loc[self.current_step - 4:self.current_step, 'Open'],
            self.df.loc[self.current_step - 4:self.current_step, 'High'],
            self.df.loc[self.current_step - 4:self.current_step, 'Low'],
            self.df.loc[self.current_step - 4:self.current_step, 'Close'],
            self.df.loc[self.current_step - 4:self.current_step, 'Volume'],
            self.df.loc[self.current_step -
                        4:self.current_step, 'Quote asset volume'],
            self.df.loc[self.current_step -
                        4:self.current_step, 'Number of trades'],
            self.df.loc[self.current_step -
                        4:self.current_step, 'Taker buy base asset volume'],
            self.df.loc[self.current_step -
                        4:self.current_step, 'Taker buy quote asset volume']
        ])
        # We append additional data
        obs = np.append(frame, [[self.balance / MAX_ACCOUNT_BALANCE,
                                 self.net_worth / self.max_net_worth,
                                 self.crypto_held / MAX_CRYPTO,
                                 self.bnb_usdt_held / self.bnb_usdt_held_start,
                                 0]],
                                axis=0)

        return obs

    def _take_action(self, action):
        # Set the current price to a random price between open and close
        current_price = random.uniform(
            self.df.loc[self.current_step, 'Real open'],
            self.df.loc[self.current_step, 'Real close'])

        if action[0] > 0:
            # Buy
            crypto_bought = self.balance * action[0] / current_price
            self.bnb_usdt_held -= crypto_bought * current_price * MAKER_FEE
            self.total_fees += crypto_bought * current_price * MAKER_FEE
            self.total_volume_traded += crypto_bought * current_price
            self.balance -= crypto_bought * current_price
            self.crypto_held += crypto_bought

        if action[0] < 0:
            # Sell
            crypto_sold = -self.crypto_held * action[0]
            self.bnb_usdt_held -= crypto_sold * current_price * TAKER_FEE
            self.total_fees += crypto_sold * current_price * TAKER_FEE
            self.total_volume_traded += crypto_sold * current_price
            self.balance += crypto_sold * current_price
            self.crypto_held -= crypto_sold

        self.net_worth = self.balance + self.crypto_held * current_price + self.bnb_usdt_held

        if self.net_worth > self.max_net_worth:
            self.max_net_worth = self.net_worth

    def step(self, action, end=True):
        # Execute one time step within the environment
        self._take_action(action)

        self.current_step += 1

        # Calculus of the reward
        profit = self.net_worth - (INITIAL_ACCOUNT_BALANCE +
                                   BNBUSDTHELD)

        profit_percent = profit / (INITIAL_ACCOUNT_BALANCE +
                                   BNBUSDTHELD) * 100

        benchmark_profit = (self.df.loc[self.current_step, 'Real open'] /
                            self.df.loc[self.start_step, 'Real open'] -
                            1) * 100

        diff = profit_percent - benchmark_profit
        reward = np.sign(diff) * (diff)**2

        # A single episode can last a maximum of MAX_STEPS steps
        if self.current_step >= MAX_STEPS + self.start_step:
            end = True
        else:
            end = False

        done = self.net_worth <= 0 or self.bnb_usdt_held <= 0 or end

        if done and end:
            self.episode_reward = reward
            self._render_episode()
            self.graph_profit.append(profit_percent)
            self.graph_benchmark.append(benchmark_profit)
            self.graph_reward.append(reward)
            self.episode += 1

        obs = self._next_observation()

        # {} needed because gym wants 4 args
        return obs, reward, done, {}

    def render(self, print_step=False, graph=False, *args):
        profit = self.net_worth - (INITIAL_ACCOUNT_BALANCE +
                                   BNBUSDTHELD)

        profit_percent = profit / (INITIAL_ACCOUNT_BALANCE +
                                   BNBUSDTHELD) * 100

        benchmark_profit = (self.df.loc[self.current_step, 'Real open'] /
                            self.df.loc[self.start_step, 'Real open'] -
                            1) * 100

        if print_step:
            print("----------------------------------------")
            print(f'Step: {self.current_step}')
            print(f'Balance: {round(self.balance, 2)}')
            print(f'Crypto held: {round(self.crypto_held, 2)}')
            print(f'Fees paid: {round(self.total_fees, 2)}')
            print(f'Volume traded: {round(self.total_volume_traded, 2)}')
            print(f'Net worth: {round(self.max_net_worth, 2)}')
            print(f'Max net worth: {round(self.max_net_worth, 2)}')
            print(f'Profit: {round(profit_percent, 2)}% ({round(profit, 2)})')
            print(f'Benchmark profit: {round(benchmark_profit, 2)}')

        # Plot the graph of the reward
        if graph:
            fig = plt.figure()
            fig.suptitle('Training graph')

            high = plt.subplot(2, 1, 1)
            high.set(ylabel='Gain')
            plt.plot(self.graph_profit, label='Bot profit')
            plt.plot(self.graph_benchmark, label='Benchmark profit')
            high.legend(loc='upper left')

            low = plt.subplot(2, 1, 2)
            low.set(xlabel='Episode', ylabel='Reward')
            plt.plot(self.graph_reward, label='reward')

            plt.show()

        return profit_percent, benchmark_profit

    def _render_episode(self, filename='render/render.txt'):
        file = open(filename, 'a')
        file.write('-----------------------\n')
        file.write(f'Episode numero: {self.episode}\n')
        file.write(f'Profit: {round(self.render()[0], 2)}%\n')
        file.write(f'Benchmark profit: {round(self.render()[1], 2)}%\n')
        file.write(f'Reward: {round(self.episode_reward, 2)}\n')
        file.close()

In [ ]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

import pandas as pd
import os

df = pd.read_csv('data/BTCUSDT.csv', index_col=0)

env = DummyVecEnv([lambda: CryptoEnv(df)])

# Instanciate the agent
model = PPO2(MlpPolicy, env, gamma=1, learning_rate=0.01, verbose=0)

# Train the agent
total_timesteps = int(os.getenv('TOTAL_TIMESTEPS', 500000))
model.learn(total_timesteps)

# Render the graph of rewards
env.render(graph=True)

# Save the agent
# model.save('PPO2_CRYPTO')

# Trained agent performence
obs = env.reset()
env.render()
for i in range(100000):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render(print_step=True)